# Modelo de Análisis de Sentimiento

## 1. Obtención y Exploración de Datos (IIIT-5K Dataset):
- Descarguen el dataset de Kaggle.
- Investiguen su estructura. Notarán que las etiquetas (el texto de cada imagen) a menudo se encuentran en archivos .mat (formato de MATLAB). Pero tambien esta en formato CSV, pueden ignorar los .mat
- Escriban los scripts para asociar correctamente cada archivo de imagen con su palabra o texto correspondiente. (Tip, el path es una columan en el CSV)

1. Descargamos el dataset de Kaggle y lo guardamos en el directorio *data*

In [5]:
import shutil
from pathlib import Path
import kagglehub

# 1) Descarga o reutiliza la ruta
downloaded = kagglehub.dataset_download("prathmeshzade/iiit5k-words")
src = Path(downloaded)

# 2) Definimos destino = carpeta padre del cwd
project_root = Path.cwd().parent           # ← aquí la carpeta “padre”
dst = project_root / "data"                # opcional: lo metemos en un subfolder “data”
dst.mkdir(exist_ok=True)

# 3) Copiamos CSVs y subcarpetas de src → dst
for item in src.iterdir():
    target = dst / item.name
    if item.is_dir():
        shutil.copytree(item, target, dirs_exist_ok=True)
    elif item.suffix.lower() == ".csv":
        shutil.copy2(item, target)

print("Contenido copiado a:", dst.resolve())


Contenido copiado a: C:\Users\alfni\OneDrive\Desktop\IA\Proyecto_2\data
